In [ ]:
! git clone https://github.com/Trest-team/Emotion_Classifier.git

Cloning into 'Emotion_Classifier'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 135 (delta 64), reused 100 (delta 32), pack-reused 0
Receiving objects: 100% (135/135), 704.50 KiB | 18.06 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [ ]:
!pip3 install -r Emotion_Classifier/requirements.txt
!pip3 install datasets

     |████████████████████████████████| 1.8MB 28.7MB/s 
     |████████████████████████████████| 819kB 35.9MB/s 
     |████████████████████████████████| 2.5MB 39.7MB/s 
     |████████████████████████████████| 174kB 53.1MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 143kB 50.9MB/s 
     |████████████████████████████████| 829kB 27.5MB/s 
     |████████████████████████████████| 10.6MB 15.7MB/s 
     |████████████████████████████████| 235kB 44.0MB/s 
     |████████████████████████████████| 122kB 41.5MB/s 
     |████████████████████████████████| 901kB 41.1MB/s 
     |████████████████████████████████| 3.3MB 27.5MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 1.3MB 36.9MB/s 
     |████████████████████████████████| 296kB 35.8MB/s 
     |████████████████████████████████| 143kB 51.3MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=df1c3ae6

In [ ]:
%cd /content/Emotion_Classifier

/content/Emotion_Classifier


In [ ]:
import sys
sys.path.append('/content/Emotion_Classifier/model')
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/Emotion_Classifier/model']


In [ ]:
import wandb
import pytorch_lightning
import transformers

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init()

wandb: Currently logged in as: jminj (use `wandb login --relogin` to force relogin)


In [ ]:
!CUDA_VISIBLE_DEVICES=0 python model/trainer_ver2.py --gpus 2 --max_epochs 8

2021-07-05 03:25:13.207261: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import ElectraTokenizerFast, ElectraTokenizer, ElectraForSequenceClassification
from transformers import Trainer, TrainingArguments
from tokenizers.processors import TemplateProcessing
from transformers import EarlyStoppingCallback
from datasets import load_metric

U_TKN = '<usr>'
S_TKN = '<sys>'
BOS = '<s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '[PAD]'

class ClassifierDataset(Dataset):
  def __init__(self, dataset_path):
    super().__init__()
    
    self.data = pd.read_csv(dataset_path, sep = '\t')
    
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, idx):
    sentence, label = self.data.iloc[idx]

    return {'sentence' : sentence, 'label' : label}


# Trainer의 data_collator와 동일
class Classifier_Collator(object): 
  # def __init__(self, vocab_path, merge_file, max_len = 32):
  def __init__(self, max_len = 32):
    super().__init__()
    self.first = True
    self.q_token = U_TKN
    self.a_token = S_TKN
    self.sent_token = SENT
    self.bos = BOS
    self.eos = EOS
    self.mask = MASK
    self.pad = PAD
    self.max_len = max_len
    self.tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-discriminator")
    
    # self.tokenizer.post_process = TemplateProcessing(
    #   single = '<s> $A </s>',
    #   special_tokens = [
    #     ("<s>", self.tokenizer.convert_tokens_to_ids(self.bos)),
    #     ("</s>", self.tokenizer.convert_tokens_to_ids(self.eos)),
    #   ],
    # )

  def masking(self, data):
    tok_data = self.tokenizer.tokenize(data)
    if len(tok_data) < (self.max_len - 2):
      for i in range((self.max_len - 2) - len(tok_data)):
        tok_data.append('[PAD]')
    elif len(tok_data) > (self.max_len - 2):
      tok_data = tok_data[:self.max_len - 1]

    return tok_data

  def __call__(self, sequences):
    sentences = [self.tokenizer.decode(self.tokenizer.convert_tokens_to_ids(self.masking(sequence['sentence']))) for sequence in sequences]
    labels = [sequence['label'] for sequence in sequences]

    toked_sentences = self.tokenizer(text = sentences, return_tensors = 'pt', padding = True, truncation = True, max_length = self.max_len)
    # print(toked_sentences)
    # for i in range(len(toked_sentences)):
    #   print('_-------------------')
    #   print(len(toked_sentences[i]))
    #   print(toked_sentences[i].ids)

    return {'input_ids' : toked_sentences.input_ids, 'labels' : torch.tensor(labels)}

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels = 7)

args = TrainingArguments(
    output_dir = "output",
    evaluation_strategy = "steps",
    eval_steps = 500,
    per_device_train_batch_size = 5,
    per_device_eval_batch_size = 5,
    num_train_epochs = 4,
    seed = 2000,
    load_best_model_at_end = True,
)

train_datasets = ClassifierDataset('/content/drive/MyDrive/dataset/emotion_dataset_mecab_merge_shuf_train.tsv')
val_datasets = ClassifierDataset('/content/drive/MyDrive/dataset/emotion_dataset_mecab_merge_shuf_val.tsv')

train_collator = Classifier_Collator()

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_datasets,
    data_collator = train_collator,
    eval_dataset = val_datasets,
    compute_metrics = compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

***** Running training *****
  Num examples = 24999
  Num Epochs = 4
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 20000


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Accuracy
500,1.719700,1.753668,0.289400


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 5
Saving model checkpoint to /content/drive/MyDrive/trest_checkpoint/checkpoint-500
Configuration saved in /content/drive/MyDrive/trest_checkpoint/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/trest_checkpoint/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 5


In [ ]:
test_data = ClassifierDataset('/content/drive/MyDrive/dataset/emotion_dataset_mecab_merge_shuf_test.tsv')

model_path = "output/checkpoint-50000"
model = ElectraForSequenceClassification.from_pretrained(model_path, num_labels=7)

test_trainer = Trainer(model)
raw_pred, _, _ = test_trainer.predict(test_dataset)

y_pred = np.argmax(raw_pred, axis=1)

In [ ]:
###
from transformers import ElectraTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained('facebook/bart-large', bos_token = '<s>', eos_token = '</s>', pad_token = '<pad>')
tokenizer2 = ElectraTokenizerFast.from_pretrained('monologg/koelectra-small-v3-discriminator', bos_token = '<s>', eos_token = '</s>', pad_token = '<pad>')
print(tokenizer2.convert_ids_to_tokens(35002))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<pad>
